In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_colwidth', None)

In [3]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_colwidth', None)

In [4]:
def bc(df_s1,df_s2):
    diff = abs(df_s1 - df_s2)
    sm = df_s1 + df_s2
    bc = diff.sum()/sm.sum()
    return bc 

In [ ]:
# Keep a set for all unique barcodes in the top 50 for each file
all_barcodes = set()
#barcode indel combo
files_barcode = [f'./Step6_Barcode_Norm/{f}' for f in listdir('./Step6_Barcode_Norm/') if isfile(join('./Step6_Barcode_Norm/', f))]
for file_barcode in files_barcode:
    print(file_barcode[file_barcode.rfind('/')+1:])
    # Read in the barcode file as a dataframe
    # The first row is the header. First column is 'barcode', second column is 'count', third column is 'normalized'
    df_barcode = pd.read_csv(file_barcode, sep='\t', header=0)
    # Set the index to be the barcode
    df_barcode = df_barcode.set_index('barcode')
    # Get the top 50 barcodes
    df_barcode_top = df_barcode.nlargest(50, 'normalized')
    # Add the barcodes to the set
    all_barcodes.update(df_barcode_top.index)

In [ ]:
import math

# Get all file names in Step8
samples = [f[:f.find('.fastq')] for f in listdir('./Step8/') if isfile(join('./Step8/', f)) and f.find("R1") != -1]
samples.sort()

# Read in the files and create frequency tables
freq_tables = {}
for id in samples:
    barcode_indel_frq = pd.read_csv(f'./Step8/{id}.fastq', header=None).value_counts()
    barcode_indel_frq = barcode_indel_frq/len(pd.read_csv(f'./Step5/{id}.fastq', header=None).index)*1000000
    barcode_indel_frq.index = barcode_indel_frq.index.map(lambda x: x[0])
    # Go through the index and remove any entries that don't have barcodes in the top 50
    for index in barcode_indel_frq.index:
        # Get barcode from index. It is the first 5 characters
        barcode = index[:5]
        if barcode not in all_barcodes:
            barcode_indel_frq = barcode_indel_frq.drop(index)
    freq_tables[id] = barcode_indel_frq

frames = []
names = []

for percent in range(20, 25, 5):
    names.append(f'{percent}%')

    # Make matrix of 0's using the index of barcode_indel_frq1_filtered_r1
    matrix_r1 = pd.DataFrame(0, index=samples, columns=samples)

    for id in matrix_r1.index:
        for col in matrix_r1.columns:
            # Get the frequency tables for each sample
            barcode_indel_frq1 = freq_tables[id]
            barcode_indel_frq2 = freq_tables[col]

            # Filter based on percent
            barcode_indel_frq1_filtered_r1 = barcode_indel_frq1[0:math.floor(len(barcode_indel_frq1)*percent*0.01)]
            barcode_indel_frq2_filtered_r1 = barcode_indel_frq2.loc[barcode_indel_frq1_filtered_r1.index.intersection(barcode_indel_frq2.index)]

            # Add 0 to the missing values
            only_frq1 = barcode_indel_frq1_filtered_r1.index.difference(barcode_indel_frq2_filtered_r1.index)
            new_index = pd.Index([index for index in barcode_indel_frq2_filtered_r1.index] + [index for index in only_frq1])
            barcode_indel_frq2_filtered_r1 = pd.Series(np.append(barcode_indel_frq2_filtered_r1.values,np.zeros(len(only_frq1))),index=new_index)
            barcode_indel_frq2_filtered_r1 = barcode_indel_frq2_filtered_r1.reindex(barcode_indel_frq1_filtered_r1.index)

            # Calculate the BC distance
            matrix_r1.loc[id, col] = bc(barcode_indel_frq1_filtered_r1,barcode_indel_frq2_filtered_r1)

    frames.append(matrix_r1)